# Codeasy Task - YOLOv8

To start with, install the [Ultralytics](https://github.com/ultralytics/ultralytics) and [Fiftyone](https://docs.voxel51.com/getting_started/install.html).

In [ ]:
!pip install fiftyone ultralytics

After succesfully installing the required packages, import the necessary libraries.

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import numpy as np
import os

From ultralytics, YOLOv8 model will be imported, downloaded, in this case that is yolov8n model.

In [ ]:
from ultralytics import YOLO

detection_model = YOLO("yolov8n.pt")

As the data that will be used is not in format of YOLOv8 (YOLOv5) requirements, custom function that uses Fiftyone export function will be use to export it in the desired format.

In [4]:
# custom function which exports the data in yolo format
def export_yolo_data(
    samples,
    export_dir,
    classes,
    label_field = "ground_truth",
    split = None
    ):

    if type(split) == list:
        splits = split
        for split in splits:
            export_yolo_data(
                samples,
                export_dir,
                classes,
                label_field,
                split
            )
    else:
        if split is None:
            split_view = samples
            split = "val"
        else:
            split_view = samples.match_tags(split)

        split_view.export(
            export_dir=export_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            label_field=label_field,
            classes=classes,
            split=split
        )

To access the data that will be used in this project, library Fiftyone.zoo will be used and its load_zoo_dataset function which has the Open Images v7 dataset. While the dataset contains millions of pictures, for the training of the model that will be used in this project, only 2 classes will be needed, that being Owl and Sheep. To only get the pictures that contain the desired classes, in the classes variable we insert the desired classes and after map the dataset adding the "ground_truth" and giving the classes of "Owl" and "Sheep" a new class name of "owl" and "sheep". Name of the class can be change to anything user wants as the name.

In [ ]:
# creating the custom dataset
custom_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    classes = ["Owl", "Sheep"],
    only_matching=True,
    label_types="detections"
).map_labels(
    "ground_truth",
    {"Owl": "owl",
     "Sheep": "sheep"
     }
)

So the dataset doesn't get deleted, using persisent = True on the dataset will keep it even if the database is shut down.

In [6]:
custom_dataset.persistent = True

After making the dataset that contains the relevant pictures, next task is to split the dataset to train set and validation set, in this case in ratio of 0.8 and 0.2. First the tags are deleted and after that random split is called so the data is randomized in each set giving the model better representaion of data. After that, already mentioned function which exports the sets in YOLO format is called and custom_train dataset is made with only 2 specified classes and train/val split.

In [ ]:
import fiftyone.utils.random as fornd

classes = ["owl", "sheep"]

## delete existing tags to start fresh
custom_dataset.untag_samples(custom_dataset.distinct("tags"))

## split into train and val
fornd.random_split(
    custom_dataset,
    {"train": 0.8, "val": 0.2}
)

## export in YOLO format
export_yolo_data(
    custom_dataset,
    "custom_train",
    classes,
    split = ["train", "val"]
)

After the dataset to train model is ready, using the !yolo command and attributes such as:
*   task - what model needs to do
*   mode - in which mode the model is set
*   model - which model is used to train
*   data - specifies the path to the dataset configuration file
*   epochs - number of training epochs, in this example dataset will be passed through the model 2 times during training.
*   imgsz - sets the input image size for training, in this exampled it is set to 640x640 pixels
*   batch - specifies how many samples, images will be processed in each iteration or batch during training, in this example it is set to 16 which means that the model will process 16 images in parallel before updating its weights based on the computed gradients.







In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data=custom_train/dataset.yaml epochs=2 imgsz=640 batch=16

After training is complete, model that performed the best on validation set is saved as the best.pt as well as the last.pt model and are ready to be used or further trained again to get better results.

In [10]:
# saving the best.pt model in variable model
model = YOLO('/content/runs/detect/train/weights/best.pt')

As the Google Colab deletes the files after session, exporting the model and saving it on your local computer or cloud server will allow for further uses even after this session is over. In this project model will be exported in default settings, further information on exporting can be found [here](https://docs.ultralytics.com/modes/export/#export-formats).

In [ ]:
# exporting model
model.export()

Model can be tested on a simple picture using the !yolo command and atributes such as:


*   model - model used to predict
*   source - specifies the path to the image/video source

In [ ]:
!yolo predict model="/content/runs/detect/train/weights/best.pt" source='/content/test2.jpeg'

Test result can be accessed in the runs/detect/predict folder.